# Seldon Core Setup

## Prerequisites

You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [A running Kubernetes cluster with kubectl authenticated](https://kubernetes.io/docs/setup/)
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core>=0.2.6.1```)
 - [Helm client](https://helm.sh/)

## Setup Cluster

In [4]:
!kubectl create namespace seldon

namespace/seldon created


In [5]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


## Install Seldon Core

Follow the [Seldon Core Install documentation](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html).

In [ ]:
!kubectl create namespace seldon-system

### If using Ambassador:

In [ ]:
!helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set ambassador.enabled=true --set usageMetrics.enabled=true --namespace seldon-system

### If using Istio:

In [ ]:
!helm install seldon-core seldon-core-operator --repo https://storage.googleapis.com/seldon-charts --set istio.enabled=true --set usageMetrics.enabled=true --namespace seldon-system

Check all services are running before proceeding.

### Wait for rollout to finish

In [ ]:
!kubectl rollout status deploy/seldon-controller-manager -n seldon-system

## Install Ingress

### Ambassador

[Ambassador install](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html#install-ambassador)

**Note:** There are reported gRPC issues with ambassador (see https://github.com/SeldonIO/seldon-core/issues/473).

In [1]:
!helm repo add stable https://kubernetes-charts.storage.googleapis.com/

"stable" has been added to your repositories


In [2]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "seldon-staging" chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈ Happy Helming!⎈ 


In [6]:
!helm install ambassador stable/ambassador --set crds.keep=false

manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
manifest_sorter.go:175: info: skipping unknown hook: "crd-install"
NAME: ambassador
LAST DEPLOYED: Wed Feb 19 15:02:45 2020
NAMES

Check all services are running before proceeding.

In [4]:
!kubectl rollout status deployment.apps/ambassador

Waiting for deployment "ambassador" rollout to finish: 0 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 1 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


In [ ]:
!kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080

Forwarding from 127.0.0.1:8003 -> 8080
Forwarding from [::1]:8003 -> 8080
Handling connection for 8003
Handling connection for 8003
Handling connection for 8003
Handling connection for 8003
Handling connection for 8003
Handling connection for 8003


*or*

### Istio

[Istio install](https://docs.seldon.io/projects/seldon-core/en/latest/workflow/install.html#install-istio-ingress-gateway)

**Note:** Remember to add ```--set istio.enabled=true``` flag when installing Seldon Core with Istio Ingress.